In [1]:
def drop_na_cols(df):
    # Calculate the threshold for NaN values
    threshold = len(df) * 0.75
    percent_null = df.isnull().sum() / len(df) * 100

    cols_to_drop = percent_null[percent_null > 50].index
    num_cols_dropped = len(df.columns) - len(cols_to_drop)
    print(f'Dropping {num_cols_dropped} cols, columns to drop: {cols_to_drop.tolist()}')
    # Drop columns exceeding the threshold
    df = df.drop(cols_to_drop,axis=1 )
    return df


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta, date


# Set up API credentials and parameters
api_key = '16AI00EAgNeGSZqgQKrLc0GUf8fVhDaa'
base_url = 'https://app.ticketmaster.com/discovery/v2/events.json'
city = 'Austin'
state = 'TX'
date_range = f"{date.today()},{date.today() + timedelta(days=90)}"
params = {
    'apikey': api_key,
    'city': city,
    'stateCode': state,
    'start_date': datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
    'end_date': (datetime.now() + timedelta(days=10)).strftime("%Y-%m-%dT%H:%M:%SZ"),
    'stateCode':'TX',
    'preferredCountry':'us',
    'size': 50 , # Adjust the number of results per page as per your needs
    # 'page': 0
}

# Send API requests and fetch all pages of data
all_events = []
page = 0

while True:
    params['page'] = page
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if '_embedded' in data and 'events' in data['_embedded']:
        act_page = data['page']['number']
        print(f'Current Actual Page: {act_page}')
        events = data['_embedded']['events']
        all_events.extend(events)
        
        if 'page' in data and 'totalPages' in data['page'] and data['page']['number'] < data['page']['totalPages'] - 1:
            page += 1
            print(f'Incremented Page # to: {page}')
        else:
            break
    else:
        print(f'Page #: {page}, Incorrect response format, terminating')
        print(data)
        break


# Create a dataframe
df = pd.DataFrame(all_events)
# Drop columns that are mostly NaNs
df = drop_na_cols(df)

#Drop Unnecessary Columns
df2 = df.drop(columns=['images','seatmap','accessibility','ageRestrictions','test','_links'])


# Print the dataframe
# print(df)



In [ ]:
df2 = df
df2
df2.info()

In [19]:

def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude:
                    flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out


def flatten_and_drop(original_df, cols_to_flatten):
    merged_df = original_df.copy()  # Initialize merged_df with the original DataFrame

    for col in cols_to_flatten:
        # Apply flatten_json function to every row in the DataFrame
        flattened_data = original_df[col].apply(flatten_json)

        # Convert the flattened data to a DataFrame
        flattened_df = pd.DataFrame.from_records(flattened_data)

        # Calculate the threshold for NaN values
        threshold = len(flattened_df) * 0.75

        # Drop columns exceeding the threshold
        flattened_df = flattened_df.dropna(axis=1, thresh=threshold)

        # Concatenate the flattened DataFrame with the merged DataFrame
        merged_df = pd.concat([merged_df, flattened_df], axis=1)

    #combine list of columns to drop 
    prefix = 'attractions_0_images'

    #Combine to drop all at once 

    columns_to_delete = cols_to_flatten + merged_df.columns[merged_df.columns.str.startswith(prefix)].tolist()
    # Drop the original columns from the DataFrame
    merged_df.drop(columns=columns_to_delete, inplace=True)

    pattern = '_0'
    #get col names in list 
    column_names = merged_df.columns.tolist()
    #use list comprehension to replace pattern in col names
    new_column_names = [name.replace(pattern, '') for name in column_names]
    #update the col names in the dataframe
    merged_df.columns = new_column_names
    return merged_df



# Apply the flatten_and_drop function to your DataFrame
df3 = flatten_and_drop(df2, ['_embedded','dates','sales','classifications','priceRanges'])

##add the price ranges back in  
price_data = df2['priceRanges'].apply(flatten_json)
price_df = pd.DataFrame.from_records(price_data)


df3 = pd.concat([df3, price_df], axis=1)

pattern = '0_'
column_names = df3.columns.tolist()
new_column_names = [name.replace(pattern, '') for name in column_names]
df3.columns = new_column_names


In [ ]:
df2

In [20]:
### decide to only keep these 
keep_cols = ['name',
'type',
'id',
'url',
'venues_name',
'venues_id',
'venues_postalCode',
'venues_timezone',
'venues_city_name',
'venues_state_name',
'venues_state_stateCode',
'venues_country_name',
'venues_country_countryCode',
'venues_address_line1',
'venues_location_longitude',
'venues_location_latitude',
'attractions_name',
'attractions_type',
'attractions_id',
'attractions_url',
'attractions_classifications_segment_id',
'attractions_classifications_segment_name',
'attractions_classifications_genre_id',
'attractions_classifications_genre_name',
'attractions_classifications_subGenre_id',
'attractions_classifications_subGenre_name',
'start_localDate',
'status_code',
'start_localTime',
'currency',
'min',
'max',]

df3 = df3[keep_cols]

In [22]:
import pandas as pd

# Define a dictionary to map old column names to new column names
column_mapping = {
    'name': 'event_name',
    'type': 'event_type',
    'id': 'event_id',
    'url': 'event_url',
    'venues_name': 'venue_name',
    'venues_id': 'venue_id',
    'venues_postalCode': 'venue_zipcode',
    'venues_timezone': 'venues_timezone',
    'venues_city_name': 'venue_city',
    'venues_state_name': 'venue_state_full',
    'venues_state_stateCode': 'venue_state_short',
    'venues_country_name': 'venue_country_name',
    'venues_country_countryCode': 'venue_country_short',
    'venues_address_line1': 'venue_address',
    'venues_location_longitude': 'venue_longitude',
    'venues_location_latitude': 'venue_latitude',
    'attractions_name': 'attraction_name',
    'attractions_type': 'attraction_type',
    'attractions_id': 'attraction_id',
    'attractions_url': 'attraction_url',
    'attractions_classifications_segment_id': 'attraction_segment_id',
    'attractions_classifications_segment_name': 'attraction_segment_name',
    'attractions_classifications_genre_id': 'attraction_genre_id',
    'attractions_classifications_genre_name': 'attraction_genre_name',
    'attractions_classifications_subGenre_id': 'attraction_subgenre_id',
    'attractions_classifications_subGenre_name': 'attraction_subgenre_name',
    'start_localDate': 'event_start_date',
    'status_code': 'ticket_status',
    'start_localTime': 'event_start_time',
    'currency': 'currency',
    'min': 'min_price',
    'max': 'max_price'
}

# Rename the columns using the dictionary
df3 = df3.rename(columns=column_mapping)

df3.to_csv('/Users/nicburkett/Downloads/after_transformation.csv')
df3.head()

##reset the index so we can convert to json 
df3.reset_index(drop=True, inplace=True)


In [ ]:
## SET UP THE KAFKA PRODUCER 
import json 
import random 
from datetime import datetime
import time
from kafka import KafkaProducer

topic_name = 'twitter'
# Messages will be serialized as JSON 
def serializer(message):
    return json.dumps(message).encode('utf-8')

# Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=serializer
)

## Clean out the producer 
producer.flush() 

# Convert each row to JSON and send as message to Kafka
for _, row in df3.iterrows():
    # Convert the row to a dictionary with column names as keys
    data = row.to_dict()

    # Include the column names in the dictionary
    message = {col: data[col] for col in df3.columns}

    # Print the JSON message
    print(message)
    
    # Send the JSON message to the Kafka topic
    producer.send(topic_name, message)
    time.sleep(5)

# Close the Kafka producer
producer.close()

In [28]:
## empty out the `twitter` topic
producer.flush()

In [7]:
######### 
######### END 
#########

In [ ]:
#### Format the columns
# Format the start date and time
df['start.dateTime'] = pd.to_datetime(df['start.dateTime'])
df['start.dateTime'] = df['start.dateTime'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

<class 'pandas.core.indexes.base.Index'>


In [33]:
params = {
    'apikey': api_key,
    'city': 'dallas',
    'stateCode': 'tx',
    'countryCode': 'US',
    'size': 50 , # Adjust the number of results per page as per your needs
}

print(f'testing {city}')

testing Austin


In [36]:
len(df.index)

586

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta, date


# Set up API credentials and parameters
api_key = '16AI00EAgNeGSZqgQKrLc0GUf8fVhDaa'
base_url = 'https://app.ticketmaster.com/discovery/v2/events.json'

params = {
    'apikey': api_key,
    'city': 'Austin',
    'stateCode': 'TX',
    'countryCode':'US',
    'size': 50 , # Adjust the number of results per page as per your needs
    # 'page': 0
}


# Send API requests and fetch all pages of data
all_events = []
page = 0

while True:
    params['page'] = page
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if '_embedded' in data and 'events' in data['_embedded']:
        act_page = data['page']['number']
        print(f'Current Actual Page: {act_page}')
        events = data['_embedded']['events']
        all_events.extend(events)
        
        if 'page' in data and 'totalPages' in data['page'] and data['page']['number'] < data['page']['totalPages'] - 1:
            page += 1
            print(f'Incremented Page # to: {page}')
        else:
            break
    else:
        print(f'Page #: {page}, Incorrect response format, terminating')
        print(data)
        break


# Create a dataframe
df = pd.DataFrame(all_events)

price_copy = df.copy()



In [54]:
# print(f'Created dataframe showing all events in {city},{stateCode} with {len(df.columns)} columns and {len(df.index)} rows')
# Drop selected cols & columns that are mostly NaNs
df = df.drop(columns=['images','seatmap','accessibility','ageRestrictions','test','_links'])
df = drop_na_cols(df)

#Drop Unnecessary Columns
print(f'Now dataframe has {len(df.columns)} columns and {len(df.index)} rows')

# Apply the flatten_and_drop function to your DataFrame
df = flatten_and_drop(df, ['_embedded','dates','sales','classifications','priceRanges'])

Dropping 11 cols, columns to drop: ['outlets', 'promoter', 'promoters', 'ticketLimit', 'products', 'info', 'pleaseNote', 'doorsTimes', 'description']
Now dataframe has 11 columns and 586 rows


In [52]:


#Drop Unnecessary Columns
print(f'Now dataframe has {len(df2.columns)} columns and {len(df2.index)} rows')


# Apply the flatten_and_drop function to your DataFrame
df2 = flatten_and_drop(df2, ['_embedded','dates','sales','classifications','priceRanges'])

##add the price ranges back in  
price_data = price_copy['priceRanges'].apply(flatten_json)
price_df = pd.DataFrame.from_records(price_data)


df2 = pd.concat([df2, price_df], axis=1)

pattern = '0_'
column_names = df2.columns.tolist()
new_column_names = [name.replace(pattern, '') for name in column_names]
df2.columns = new_column_names

Now dataframe has 74 columns and 586 rows


KeyError: '_embedded'

In [ ]:

# Define the pattern to replace
pattern = '_0'

# Get the current column names
column_names = df3.columns.tolist()

# Replace the pattern in column names
new_column_names = [name.replace(pattern, '') for name in column_names]

# Update the column names in the DataFrame
df3.columns = new_column_names

# Output the DataFrame with updated column names
(df3.columns)

Index(['name', 'type', 'id', 'test', 'url', 'locale', 'images', 'seatmap',
       '_links', 'accessibility', 'ageRestrictions', 'ticketing',
       'venues_name', 'venues_type', 'venues_id', 'venues_test',
       'venues_locale', 'venues_postalCode', 'venues_timezone',
       'venues_city_name', 'venues_state_name', 'venues_state_stateCode',
       'venues_country_name', 'venues_country_countryCode',
       'venues_address_line1', 'venues_location_longitude',
       'venues_location_latitude', 'venues_dmas_id',
       'venues_upcomingEvents__total', 'venues_upcomingEvents__filtered',
       'venues_upcomingEventsBy_country_US', 'venues__links_self_href',
       'attractions_name', 'attractions_type', 'attractions_id',
       'attractions_test', 'attractions_url', 'attractions_locale',
       'attractions_classifications_primary',
       'attractions_classifications_segment_id',
       'attractions_classifications_segment_name',
       'attractions_classifications_genre_id',
       'att

In [ ]:
prefix = 'attractions_0_images'
columns_to_delete = df3.columns[df3.columns.str.startswith(prefix)]
columns_to_delete


def delete_columns(df,prefix):
    columns_to_delete = df.columns[df.columns.str.startswith(prefix)]
    df.drop(columns=columns_to_delete, inplace=True)
    return df



Index(['attractions_0_images_0_ratio', 'attractions_0_images_0_url',
       'attractions_0_images_0_width', 'attractions_0_images_0_height',
       'attractions_0_images_0_fallback', 'attractions_0_images_1_ratio',
       'attractions_0_images_1_url', 'attractions_0_images_1_width',
       'attractions_0_images_1_height', 'attractions_0_images_1_fallback',
       'attractions_0_images_2_ratio', 'attractions_0_images_2_url',
       'attractions_0_images_2_width', 'attractions_0_images_2_height',
       'attractions_0_images_2_fallback', 'attractions_0_images_3_ratio',
       'attractions_0_images_3_url', 'attractions_0_images_3_width',
       'attractions_0_images_3_height', 'attractions_0_images_3_fallback',
       'attractions_0_images_4_ratio', 'attractions_0_images_4_url',
       'attractions_0_images_4_width', 'attractions_0_images_4_height',
       'attractions_0_images_4_fallback', 'attractions_0_images_5_ratio',
       'attractions_0_images_5_url', 'attractions_0_images_5_width'